In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
movie=pd.read_csv("/content/drive/MyDrive/movies.csv")


In [5]:
rating=pd.read_csv("/content/drive/MyDrive/ratings.csv")

In [6]:
movie.head(5)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
import re

In [8]:
movie['year']=movie['title'].str.extract(r"\((\d{4})\)")

In [9]:
movie['title']=movie['title'].str.extract(r"^(.*?)\s\(\d{4}\)")

In [10]:
movie.head(3)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995


In [11]:
movie['year'].isnull().sum()

615

In [12]:
movie['year']=np.where(movie['year']==movie['year'].isnull(),0,movie['year'])

In [13]:
movie['genres']=movie['genres'].str.split('|')

In [14]:
movie['genres']=movie['genres'].apply(lambda x: ' '.join(x))

In [15]:
movie['genres']=movie['genres']+' '+movie['year']

In [16]:
movie.drop(columns=['year'],inplace=True)

In [17]:
movie['genres']=movie['genres'].str.lower()

In [18]:
movie.isnull().sum()

,0
movieId,0
title,616
genres,615


In [19]:
movie.head(2)

,movieId,title,genres
0,1,Toy Story,adventure animation children comedy fantasy 1995
1,2,Jumanji,adventure children fantasy 1995


In [20]:
movie.dropna(inplace=True)

In [21]:
rating.head(5)

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [22]:
rating.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [23]:
rating.shape

(32000204, 4)

In [24]:
rating.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [25]:
rating.groupby('rating')[['userId', 'movieId', 'rating', 'timestamp']].count()

,userId,movieId,rating,timestamp
rating,,,,
0.5,525132,525132,525132,525132
1.0,946675,946675,946675,946675
1.5,531063,531063,531063,531063
2.0,2028622,2028622,2028622,2028622
2.5,1685386,1685386,1685386,1685386
3.0,6054990,6054990,6054990,6054990
3.5,4290105,4290105,4290105,4290105
4.0,8367654,8367654,8367654,8367654
4.5,2974000,2974000,2974000,2974000


In [26]:
rating=rating[['userId', 'movieId', 'rating']]

In [27]:
rating.head(5)

,userId,movieId,rating
0,1,17,4.0
1,1,25,1.0
2,1,29,2.0
3,1,30,5.0
4,1,32,5.0


In [28]:
df=pd.merge(movie,rating,on='movieId')

In [28]:
df.head(5)

,movieId,title,genres,userId,rating
0,1,Toy Story,adventure animation children comedy fantasy 1995,10,2.5
1,1,Toy Story,adventure animation children comedy fantasy 1995,11,3.0
2,1,Toy Story,adventure animation children comedy fantasy 1995,17,4.0
3,1,Toy Story,adventure animation children comedy fantasy 1995,19,3.0
4,1,Toy Story,adventure animation children comedy fantasy 1995,20,5.0


In [29]:
df=df[['movieId','userId','rating','title','genres']]

In [30]:
for i in df.columns:
  print(df[i].isnull().sum())

0
0
0
0
0


In [31]:
df.shape

(31965050, 5)

In [32]:
total_movie=df.groupby('movieId')['rating'].agg('count')
total_user=df.groupby('userId')['rating'].agg('count')

In [33]:
total_user.count()

200948

In [34]:
df=df.loc[total_user[total_user>10].index,:]

In [35]:
df.shape

(200948, 5)

## **Collaborative Filtering**
```
# This is formatted as code
```



In [36]:
new_data=df.pivot(index='movieId',columns='userId',values='rating')

In [37]:
new_data

userId,3,5,10,11,13,17,19,20,23,24,...,200938,200939,200940,200941,200942,200943,200944,200946,200947,200948
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,3.0,NaN,4.0,3.0,5.0,3.0,4.0,...,4.5,NaN,NaN,NaN,NaN,3.5,4.0,NaN,4.0,5.0
2,3.5,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,1.0,2.5,NaN,NaN,NaN,1.5
3,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,3.0,5.0,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.5,NaN,4.0,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
new_data.fillna(0,inplace=True)

In [39]:
new_data.head(5)

userId,3,5,10,11,13,17,19,20,23,24,...,200938,200939,200940,200941,200942,200943,200944,200946,200947,200948
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,3.0,0.0,4.0,3.0,5.0,3.0,4.0,...,4.5,0.0,0.0,0.0,0.0,3.5,4.0,0.0,4.0,5.0
2,3.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,1.0,2.5,0.0,0.0,0.0,1.5
3,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
from scipy.sparse import csr_matrix

In [41]:
crs_data=csr_matrix(new_data)
new_data.reset_index(inplace=True)

In [42]:
print(crs_data)

  (0, 3)	3.0
  (0, 5)	4.0
  (0, 6)	3.0
  (0, 7)	5.0
  (0, 8)	3.0
  (0, 9)	4.0
  (0, 10)	4.0
  (0, 13)	5.0
  (0, 14)	4.0
  (0, 16)	3.0
  (0, 17)	1.0
  (0, 19)	5.0
  (0, 20)	4.0
  (0, 23)	3.5
  (0, 24)	4.0
  (0, 25)	4.0
  (0, 26)	4.0
  (0, 27)	3.0
  (0, 29)	5.0
  (0, 30)	5.0
  (0, 31)	2.0
  (0, 32)	4.0
  (0, 33)	4.0
  (0, 36)	4.5
  (0, 37)	4.5
  :	:
  (9, 79631)	3.0
  (9, 79632)	3.0
  (9, 79634)	3.0
  (9, 79639)	4.0
  (9, 79645)	4.0
  (9, 79647)	4.0
  (9, 79648)	4.0
  (9, 79649)	4.0
  (9, 79650)	4.0
  (9, 79653)	4.0
  (9, 79655)	4.0
  (9, 79658)	3.0
  (9, 79660)	3.0
  (9, 79661)	4.5
  (9, 79663)	5.0
  (9, 79664)	3.0
  (9, 79669)	3.0
  (9, 79670)	3.5
  (9, 79671)	3.0
  (9, 79673)	4.0
  (9, 79675)	3.0
  (9, 79679)	3.5
  (9, 79682)	3.0
  (9, 79683)	2.5
  (9, 79684)	2.5


In [43]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
knn.fit(crs_data)



NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [53]:
def get_recommendation(movie_name):
    # Find the movie in the dataset
    movie_list = movie[movie['title'].str.contains(movie_name, case=False)]

    if len(movie_list):  # If movie found
        movie_idx = movie_list.iloc[0]['movieId']  # Get movieId
        movie_idx = new_data[new_data['movieId'] == movie_idx].index[0]  # Map to matrix index

        # Ensure n_neighbors does not exceed available samples
        n_neighbors = min(10, knn.n_samples_fit_)

        # Get neighbors
        distance, indices = knn.kneighbors(crs_data[movie_idx], n_neighbors=n_neighbors)

        # Prepare recommended movies
        rec_movies_indices = sorted(
            list(zip(indices.squeeze().tolist(), distance.squeeze().tolist())),
            key=lambda x: x[1]
        )[1:]  # Exclude the movie itself (distance=0)

        recommended_movies = []
        for val in rec_movies_indices:
            movie_idx = new_data.iloc[val[0]]['movieId']  # Get movieId
            idx = movie[movie['movieId'] == movie_idx].index
            recommended_movies.append({
                'Recomended Movie For You': movie.iloc[idx]['title'].values[0],
            })

        # Create DataFrame for recommendations
        df = pd.DataFrame(recommended_movies, index=range(1, len(recommended_movies) + 1))
        return df
    else:
        return "Movie not found..."




In [54]:
# Example usage
recommendations = get_recommendation('Grumpier Old Men')
print(recommendations)

      Recomended Movie For You
1  Father of the Bride Part II
2                      Sabrina
3                    Toy Story
4                         Heat
5                 Sudden Death
6                      Jumanji
7                    GoldenEye
8            Waiting to Exhale
9                 Tom and Huck


In [55]:
!pip install gradio


In [56]:
import gradio as gr

def recommend_movies(movie_name):
    df = get_recommendation(movie_name)
    if isinstance(df, pd.DataFrame):
        return df.to_string(index=False)
    else:
        return df
app = gr.Interface(
    fn = recommend_movies,
    inputs = "text",
    outputs = "text",
    title = "Movie Recommendation System",
    description = "Enter a movie name to get a list of recommended movies"
)

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://890e09ddd37627b119.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
